## Import Required Libraries

## Define Notification Types

## Simulate User Events

## Implement Notification System

## Display Notifications

## Testing the Notification System

In [ ]:
# Mark a few notifications as read
notification_ids = [n.id for n in notification_system.notifications[:3]]
for notification_id in notification_ids:
    notification_system.mark_as_read(notification_id)

# Display updated notifications
display_notifications(notification_system)

## Interactive Notification Demo

## Conclusion

This notebook implements a basic notification system that can:
- Create different types of notifications (friend requests, messages, forum responses)
- Track read/unread status
- Display notifications in a user-friendly format
- Simulate user events that generate notifications

The system can be extended for real applications by:
1. Integrating with a database to persist notifications
2. Adding user authentication
3. Implementing real-time notifications with websockets
4. Adding notification preferences and filtering options

## Import Required Libraries

We'll import necessary libraries for working with databases, datetime handling, and notification display.

## Define Notification Types

We'll define the different types of notifications our system will handle. The main types are:

1. **Friend Requests** - When a user sends a friend request
2. **Messages** - When a user receives a new message
3. **Forum Responses** - When someone responds to a user's forum thread

## Simulate User Events

Let's create some mock data and simulate user events that would trigger notifications.

## Implement Notification System

Now we'll implement the core notification system that will handle storing, retrieving, and managing notifications.

## Display Notifications

Let's create functions to display notifications in a user-friendly format. In a real-world application, this would be HTML templates rendered by the server or JavaScript components in the frontend.

In [ ]:
# Create a visualization of notification types for all users
def visualize_notification_stats():
    """Create visualization of notification statistics across users"""
    data = []
    for user_id in notification_system.notifications:
        stats = notification_system.get_notification_stats(user_id)
        
        # Find user name (in a real app would be from the database)
        user_name = next((u['name'] for u in users if u['id'] == user_id), f"User {user_id}")
        
        # Extract counts for each type
        for n_type, counts in stats["types"].items():
            type_name = n_type.replace('_', ' ').title() # Convert to display name
            data.append({
                "user": user_name,
                "type": type_name,
                "total": counts["total"],
                "unread": counts["unread"]
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Create a stacked bar chart if we have data
    if not df.empty:
        plt.figure(figsize=(12, 8))
        
        # Plot total counts
        df_pivot = df.pivot(index='user', columns='type', values='total')
        df_pivot.fillna(0, inplace=True)
        
        # Set a nice color palette
        colors = ['#1890ff', '#52c41a', '#fa8c16', '#f5222d']
        
        ax = df_pivot.plot(kind='bar', stacked=True, figsize=(10, 6), 
                          color=colors[:len(df_pivot.columns)])
        
        plt.title('Notification Types by User', fontsize=16)
        plt.xlabel('User', fontsize=12)
        plt.ylabel('Number of Notifications', fontsize=12)
        plt.legend(title='Notification Type', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(axis='y', alpha=0.3)
        
        # Add data labels to the stacked bars
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', label_type='center')
        
        plt.tight_layout()
        plt.show()
        
        # Create a second chart showing read vs unread
        plt.figure(figsize=(12, 8))
        
        # Group by user and calculate read/unread
        user_totals = df.groupby('user').agg({'total': 'sum', 'unread': 'sum'})
        user_totals['read'] = user_totals['total'] - user_totals['unread']
        
        # Plot read vs unread
        user_totals[['read', 'unread']].plot(kind='bar', stacked=True, 
                                           color=['#52c41a', '#f5222d'])
        
        plt.title('Read vs Unread Notifications by User', fontsize=16)
        plt.xlabel('User', fontsize=12)
        plt.ylabel('Number of Notifications', fontsize=12)
        plt.legend(title='Status')
        plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("No notification data available for visualization")

# Visualize notification stats
visualize_notification_stats()

In [ ]:
def create_notification_dashboard(user_id):
    """Create an interactive HTML dashboard for notifications"""
    # Get user info - in real application would come from database
    user = next((u for u in users if u['id'] == user_id), {"name": f"User {user_id}", "username": f"user{user_id}"})
    stats = notification_system.get_notification_stats(user_id)
    notifications = notification_system.get_notifications(user_id)
    
    # Create main dashboard HTML
    html = f"""
    <div style="font-family: 'Segoe UI', Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
        <div style="display: flex; align-items: center; margin-bottom: 20px;">
            <div style="width: 60px; height: 60px; border-radius: 50%; background-color: #1890ff; color: white; display: flex; align-items: center; justify-content: center; font-size: 24px; margin-right: 15px;">
                {user['name'][0]}
            </div>
            <div>
                <h1 style="margin: 0; color: #333;">{user['name']}'s Notifications</h1>
                <p style="margin: 5px 0 0; color: #666;">@{user['username']}</p>
            </div>
        </div>
        
        <div style="display: flex; justify-content: space-between; margin-bottom: 20px;">
            <div style="flex: 1; background-color: #e6f7ff; padding: 15px; border-radius: 8px; margin-right: 10px; text-align: center;">
                <div style="font-size: 32px; font-weight: bold; color: #1890ff;">{stats['total']}</div>
                <div style="color: #666;">Total Notifications</div>
            </div>
            <div style="flex: 1; background-color: #fff1f0; padding: 15px; border-radius: 8px; margin-right: 10px; text-align: center;">
                <div style="font-size: 32px; font-weight: bold; color: #f5222d;">{stats['unread']}</div>
                <div style="color: #666;">Unread</div>
            </div>
            <div style="flex: 1; background-color: #f6ffed; padding: 15px; border-radius: 8px; text-align: center;">
                <div style="font-size: 32px; font-weight: bold; color: #52c41a;">{stats['total'] - stats['unread']}</div>
                <div style="color: #666;">Read</div>
            </div>
        </div>
        
        <div style="margin-bottom: 20px;">
            <h2 style="margin-top: 0; border-bottom: 1px solid #eee; padding-bottom: 10px; color: #333;">Recent Notifications</h2>
            
            <div style="display: flex; margin-bottom: 15px;">
                <div style="flex: 1;">
                    <button style="background-color: #1890ff; color: white; border: none; padding: 8px 16px; border-radius: 4px; cursor: pointer; margin-right: 10px;" onclick="console.log('Mark all as read');">Mark All as Read</button>
                    <button style="background-color: #f5f5f5; color: #333; border: 1px solid #d9d9d9; padding: 8px 16px; border-radius: 4px; cursor: pointer;" onclick="console.log('Clear all');">Clear All</button>
                </div>
                <div>
                    <select style="padding: 8px; border: 1px solid #d9d9d9; border-radius: 4px;" onchange="console.log('Filter changed')">
                        <option value="all">All Types</option>
                        <option value="friend_request">Friend Requests</option>
                        <option value="message">Messages</option>
                        <option value="forum_response">Forum Responses</option>
                    </select>
                </div>
            </div>
    """
    
    # No notifications message
    if not notifications:
        html += """
        <div style="text-align: center; padding: 40px 20px; background-color: #f5f5f5; border-radius: 8px;">
            <div style="font-size: 48px; margin-bottom: 10px;">🔔</div>
            <h3 style="margin: 0 0 10px; color: #333;">No notifications</h3>
            <p style="margin: 0; color: #666;">You're all caught up!</p>
        </div>
        """
    else:
        # Add notifications list
        html += "<div style='max-height: 400px; overflow-y: auto;'>"
        
        for notification in notifications[:5]:  # Show only the 5 most recent
            # Style based on read status
            bg_color = "#e6f7ff" if not notification.is_read else "#f8f9fa"
            border = "1px solid #91d5ff" if not notification.is_read else "1px solid #eee"
            status_indicator = "<span style='display: inline-block; width: 10px; height: 10px; border-radius: 50%; background-color: #1890ff; margin-right: 8px;'></span>" if not notification.is_read else ""
            
            # Format timestamp
            time_str = notification.timestamp.strftime("%b %d, %Y %H:%M")
            
            # Choose icon based on notification type
            icon = "👤" if notification.type == NOTIFICATION_TYPES['FRIEND_REQUEST'] else \
                   "✉️" if notification.type == NOTIFICATION_TYPES['MESSAGE'] else "💬"
            
            html += f"""
            <div style="padding: 16px; margin-bottom: 12px; border-radius: 8px; background-color: {bg_color}; border: {border}; position: relative;">
                <div style="display: flex; align-items: flex-start;">
                    <div style="font-size: 24px; margin-right: 16px;">{icon}</div>
                    <div style="flex-grow: 1;">
                        <div style="margin-bottom: 8px;">{status_indicator}{notification.get_message()}</div>
                        <div style="font-size: 12px; color: #888;">{time_str}</div>
                    </div>
                    <div>
                        <button style="background: none; border: none; cursor: pointer; color: #1890ff; margin-right: 8px;" 
                                onclick="console.log('Mark as read')">
                            {"Mark as read" if not notification.is_read else ""}
                        </button>
                        <button style="background: none; border: none; cursor: pointer; color: #ff4d4f;" 
                                onclick="console.log('Delete notification')">
                            ✕
                        </button>
                    </div>
                </div>
            </div>
            """
        
        html += "</div>"
        
        # Show view all button if we have more than 5 notifications
        if len(notifications) > 5:
            html += """
            <div style="text-align: center; margin-top: 20px;">
                <button style="background: none; border: none; color: #1890ff; cursor: pointer; font-size: 14px;" 
                        onclick="console.log('View all notifications')">
                    View all notifications
                </button>
            </div>
            """
    
    # Close main div
    html += "</div>"
    
    return HTML(html)

# Display the dashboard for user 1
display(create_notification_dashboard(1))

# Notification System Implementation

This notebook implements a notification system that handles friend requests, new messages, and forum responses.

In [ ]:
import time
import random
import datetime
import json
from IPython.display import display, HTML
import pandas as pd

In [ ]:
# Define notification types as a class for better organization
class NotificationType:
    FRIEND_REQUEST = "friend_request"
    NEW_MESSAGE = "new_message"
    FORUM_RESPONSE = "forum_response"

# Define notification structure
class Notification:
    def __init__(self, notification_id, notification_type, sender, content, timestamp=None, is_read=False):
        self.id = notification_id
        self.type = notification_type
        self.sender = sender
        self.content = content
        self.timestamp = timestamp if timestamp else datetime.datetime.now()
        self.is_read = is_read
    
    def to_dict(self):
        return {
            "id": self.id,
            "type": self.type,
            "sender": self.sender,
            "content": self.content,
            "timestamp": self.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "is_read": self.is_read
        }
    
    def __str__(self):
        return f"[{self.timestamp.strftime('%H:%M:%S')}] {self.sender}: {self.content} ({self.type})"

In [ ]:
class UserEventSimulator:
    def __init__(self):
        self.users = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank"]
        self.forum_topics = ["Tech Support", "General Discussion", "Feature Requests", "Bug Reports"]
        self.message_templates = [
            "Hey, how's it going?",
            "Did you see the latest update?",
            "Can you help me with something?",
            "Thanks for your response!",
            "I have a question about your post",
            "Great job on your project!"
        ]
        
    def generate_friend_request(self):
        sender = random.choice(self.users)
        return {
            "type": NotificationType.FRIEND_REQUEST,
            "sender": sender,
            "content": f"{sender} has sent you a friend request."
        }
    
    def generate_new_message(self):
        sender = random.choice(self.users)
        message = random.choice(self.message_templates)
        return {
            "type": NotificationType.NEW_MESSAGE,
            "sender": sender,
            "content": message
        }
    
    def generate_forum_response(self):
        sender = random.choice(self.users)
        topic = random.choice(self.forum_topics)
        return {
            "type": NotificationType.FORUM_RESPONSE,
            "sender": sender,
            "content": f"{sender} responded to your post in '{topic}'"
        }
    
    def generate_random_event(self):
        event_generators = [
            self.generate_friend_request,
            self.generate_new_message,
            self.generate_forum_response
        ]
        return random.choice(event_generators)()

## Implement Notification System

Write a function or class to handle notifications, including storing and retrieving them.

In [ ]:
class NotificationSystem:
    def __init__(self):
        self.notifications = []
        self.next_id = 1
    
    def add_notification(self, notification_type, sender, content):
        """Add a new notification to the system"""
        notification = Notification(
            notification_id=self.next_id,
            notification_type=notification_type,
            sender=sender,
            content=content
        )
        self.next_id += 1
        self.notifications.append(notification)
        return notification
    
    def get_all_notifications(self):
        """Get all notifications"""
        return self.notifications
    
    def get_unread_notifications(self):
        """Get only unread notifications"""
        return [n for n in self.notifications if not n.is_read]
    
    def mark_as_read(self, notification_id):
        """Mark a notification as read"""
        for notification in self.notifications:
            if notification.id == notification_id:
                notification.is_read = True
                return True
        return False
    
    def mark_all_as_read(self):
        """Mark all notifications as read"""
        for notification in self.notifications:
            notification.is_read = True
    
    def get_notifications_by_type(self, notification_type):
        """Get notifications filtered by type"""
        return [n for n in self.notifications if n.type == notification_type]
    
    def clear_notifications(self):
        """Clear all notifications"""
        self.notifications = []

In [ ]:
# Test the notification system
notification_system = NotificationSystem()

# Add some notifications
event = simulator.generate_friend_request()
notification_system.add_notification(event["type"], event["sender"], event["content"])

event = simulator.generate_new_message()
notification_system.add_notification(event["type"], event["sender"], event["content"])

event = simulator.generate_forum_response()
notification_system.add_notification(event["type"], event["sender"], event["content"])

# Display the notifications
for notification in notification_system.get_all_notifications():
    print(notification)

## Display Notifications

Create a mechanism to display notifications to the user, such as printing them to the console or simulating a UI.

In [ ]:
class NotificationUI:
    def __init__(self, notification_system):
        self.notification_system = notification_system
    
    def display_notifications_text(self):
        """Display notifications as text in the console"""
        notifications = self.notification_system.get_all_notifications()
        if not notifications:
            print("No notifications to display.")
            return
        
        print("\n=== Notifications ===")
        for notification in notifications:
            status = "NEW" if not notification.is_read else "Read"
            print(f"[{status}] {notification}")
        print("====================\n")
    
    def display_notifications_html(self):
        """Display notifications in a formatted HTML table"""
        notifications = self.notification_system.get_all_notifications()
        if not notifications:
            return HTML("<p>No notifications to display.</p>")
        
        # Convert notifications to DataFrame for prettier display
        df = pd.DataFrame([n.to_dict() for n in notifications])
        # Reorder columns for better presentation
        df = df[["id", "type", "sender", "content", "timestamp", "is_read"]]
        
        # Style the DataFrame for notifications
        styled_df = df.style.apply(self._style_read_status, axis=1)
        styled_df = styled_df.set_caption("Notifications")
        
        return display(styled_df)
    
    def _style_read_status(self, row):
        """Style rows based on read status"""
        if not row["is_read"]:
            return ['background-color: #e6f7ff'] * len(row)
        return [''] * len(row)
    
    def notification_count_badge(self):
        """Display a notification count badge"""
        unread_count = len(self.notification_system.get_unread_notifications())
        if unread_count > 0:
            badge_html = f"""
            <div style="display: inline-block; background-color: #ff4d4f; color: white; 
                       border-radius: 50%; width: 24px; height: 24px; text-align: center; 
                       line-height: 24px; font-weight: bold;">
                {unread_count}
            </div>
            <span style="margin-left: 10px;">New Notifications</span>
            """
            return HTML(badge_html)
        else:
            return HTML("<span>No new notifications</span>")

In [ ]:
# Initialize the UI
ui = NotificationUI(notification_system)

# Display notifications
ui.display_notifications_text()

# Display the notification badge
ui.notification_count_badge()

In [ ]:
# Display notifications in HTML format
ui.display_notifications_html()

## Simulate Real-time Notifications

This section demonstrates how the notification system would work in a real-time environment by simulating periodic notifications.

In [ ]:
def simulate_realtime_notifications(duration=10, interval=2):
    """
    Simulate real-time notifications for a specified duration
    
    Parameters:
    - duration: Total simulation time in seconds
    - interval: Time between notification checks in seconds
    """
    system = NotificationSystem()
    ui = NotificationUI(system)
    simulator = UserEventSimulator()
    
    start_time = time.time()
    end_time = start_time + duration
    
    print(f"Starting notification simulation for {duration} seconds...")
    
    while time.time() < end_time:
        # Generate a random event with 50% probability
        if random.random() > 0.5:
            event = simulator.generate_random_event()
            notification = system.add_notification(event["type"], event["sender"], event["content"])
            print(f"\nNew notification received: {notification}")
            display(ui.notification_count_badge())
        
        # Wait for the next check
        time.sleep(interval)
    
    print("\nSimulation complete. Final notification status:")
    ui.display_notifications_html()

In [ ]:
# Run the simulation for 10 seconds with checks every 2 seconds
# simulate_realtime_notifications(10, 2)

## Complete Notification System Demo

Now let's create a complete demonstration of the notification system with all the functionality.

In [ ]:
def full_demo():
    """Full demonstration of the notification system"""
    # Create a new notification system
    system = NotificationSystem()
    ui = NotificationUI(system)
    simulator = UserEventSimulator()
    
    print("=== Notification System Demo ===\n")
    
    # 1. Generate some initial notifications
    print("Generating initial notifications...")
    for _ in range(5):
        event = simulator.generate_random_event()
        system.add_notification(event["type"], event["sender"], event["content"])
    
    # 2. Display all notifications
    print("\n--- All Notifications ---")
    ui.display_notifications_html()
    
    # 3. Mark some as read
    print("\n--- Marking some notifications as read ---")
    # Mark the first and third notifications as read
    system.mark_as_read(1)
    system.mark_as_read(3)
    ui.display_notifications_html()
    
    # 4. Show the notification badge
    print("\n--- Notification Badge ---")
    display(ui.notification_count_badge())
    
    # 5. Filter notifications by type
    print("\n--- Friend Request Notifications Only ---")
    friend_requests = system.get_notifications_by_type(NotificationType.FRIEND_REQUEST)
    for notification in friend_requests:
        print(notification)
    
    # 6. Add a new notification
    print("\n--- Adding a new notification ---")
    event = simulator.generate_new_message()
    new_notif = system.add_notification(event["type"], event["sender"], event["content"])
    print(f"Added: {new_notif}")
    
    # 7. Show the final state
    print("\n--- Final Notification State ---")
    ui.display_notifications_html()
    
    print("\n=== End of Demo ===")

# Run the full demo
full_demo()

# Real-Time Notification System

This notebook implements a notification system for friend requests, new messages, and forum responses, simulating a real-time notification experience.

## Import Required Libraries

Import necessary libraries such as time and threading for simulating real-time notifications.

In [ ]:
# Import required libraries
import time
import threading
import random
import json
from IPython.display import display, clear_output, HTML
import queue
from datetime import datetime

## Define Notification Types

Create a dictionary or class to define different types of notifications (e.g., friend request, new message, forum response).

In [ ]:
# Define notification types and their properties
class NotificationType:
    FRIEND_REQUEST = "friend_request"
    NEW_MESSAGE = "new_message"
    FORUM_RESPONSE = "forum_response"

# Define notification templates with styling
NOTIFICATION_TEMPLATES = {
    NotificationType.FRIEND_REQUEST: {
        "icon": "👥",
        "color": "#4a86e8",
        "template": "{icon} <span style='color:{color};'><b>Friend Request:</b></span> {sender} wants to connect with you."
    },
    NotificationType.NEW_MESSAGE: {
        "icon": "✉️",
        "color": "#6aa84f",
        "template": "{icon} <span style='color:{color};'><b>New Message:</b></span> {sender} sent you: \"{content}\""
    },
    NotificationType.FORUM_RESPONSE: {
        "icon": "🗨️",
        "color": "#e69138",
        "template": "{icon} <span style='color:{color};'><b>Forum Response:</b></span> {sender} replied to your post in {forum}"
    }
}

# User database (simulated)
USERS = ["Alice", "Bob", "Charlie", "David", "Emma", "Frank", "Grace", "Hannah"]
FORUMS = ["General Discussion", "Technical Support", "Feature Requests", "Introductions", "Off-Topic"]

## Simulate User Events

Write functions to simulate user events like sending a friend request, a new message, or a forum response.

In [ ]:
# Create classes to simulate user events

class UserEvent:
    def __init__(self, sender, recipient):
        self.sender = sender
        self.recipient = recipient
        self.timestamp = datetime.now()
    
    def to_notification(self):
        """Convert event to a notification"""
        raise NotImplementedError("Subclasses must implement to_notification()")

class FriendRequest(UserEvent):
    def to_notification(self):
        template = NOTIFICATION_TEMPLATES[NotificationType.FRIEND_REQUEST]
        return {
            "type": NotificationType.FRIEND_REQUEST,
            "icon": template["icon"],
            "color": template["color"],
            "content": template["template"].format(
                icon=template["icon"],
                color=template["color"],
                sender=self.sender
            ),
            "sender": self.sender,
            "timestamp": self.timestamp,
            "read": False
        }

class NewMessage(UserEvent):
    def __init__(self, sender, recipient, message):
        super().__init__(sender, recipient)
        self.message = message
        
    def to_notification(self):
        template = NOTIFICATION_TEMPLATES[NotificationType.NEW_MESSAGE]
        return {
            "type": NotificationType.NEW_MESSAGE,
            "icon": template["icon"],
            "color": template["color"],
            "content": template["template"].format(
                icon=template["icon"],
                color=template["color"],
                sender=self.sender,
                content=self.message
            ),
            "sender": self.sender,
            "message": self.message,
            "timestamp": self.timestamp,
            "read": False
        }

class ForumResponse(UserEvent):
    def __init__(self, sender, recipient, forum):
        super().__init__(sender, recipient)
        self.forum = forum
        
    def to_notification(self):
        template = NOTIFICATION_TEMPLATES[NotificationType.FORUM_RESPONSE]
        return {
            "type": NotificationType.FORUM_RESPONSE,
            "icon": template["icon"],
            "color": template["color"],
            "content": template["template"].format(
                icon=template["icon"],
                color=template["color"],
                sender=self.sender,
                forum=self.forum
            ),
            "sender": self.sender,
            "forum": self.forum,
            "timestamp": self.timestamp,
            "read": False
        }

# Function to generate random events for simulation
def generate_random_event():
    event_type = random.choice([FriendRequest, NewMessage, ForumResponse])
    sender = random.choice(USERS)
    recipient = "You"  # Assuming notifications are for the current user
    
    if event_type == FriendRequest:
        return FriendRequest(sender, recipient)
    elif event_type == NewMessage:
        messages = [
            "Hey, how are you?",
            "Check out this new post!",
            "Are you coming to the meetup?",
            "I need your help with something",
            "Have you seen the latest update?"
        ]
        return NewMessage(sender, recipient, random.choice(messages))
    else:  # ForumResponse
        return ForumResponse(sender, recipient, random.choice(FORUMS))

## Implement Notification System

Develop a function or class to handle notifications, including queuing and displaying them to the user.

In [ ]:
class NotificationSystem:
    def __init__(self):
        self.notification_queue = queue.Queue()
        self.notifications = []
        self.running = False
        self.max_notifications = 10  # Maximum number of notifications to store
    
    def add_notification(self, event):
        """Add a new notification from an event"""
        notification = event.to_notification()
        self.notification_queue.put(notification)
        return notification
    
    def process_notifications(self):
        """Process notifications from the queue"""
        while not self.notification_queue.empty():
            notification = self.notification_queue.get()
            # Add to the beginning (most recent first)
            self.notifications.insert(0, notification)
            # Limit the number of stored notifications
            if len(self.notifications) > self.max_notifications:
                self.notifications.pop()
    
    def get_unread_count(self):
        """Get the count of unread notifications"""
        return sum(1 for n in self.notifications if not n.get("read", False))
    
    def mark_all_as_read(self):
        """Mark all notifications as read"""
        for notification in self.notifications:
            notification["read"] = True
    
    def mark_as_read(self, index):
        """Mark a specific notification as read"""
        if 0 <= index < len(self.notifications):
            self.notifications[index]["read"] = True
    
    def generate_html(self):
        """Generate HTML representation of notifications"""
        if not self.notifications:
            return "<p>No notifications</p>"
            
        html = "<div style='font-family: Arial, sans-serif;'>"
        html += f"<h3>Notifications ({self.get_unread_count()} unread)</h3>"
        
        for i, notification in enumerate(self.notifications):
            read_status = "" if notification.get("read", False) else "font-weight: bold; "
            time_str = notification["timestamp"].strftime("%H:%M:%S")
            
            html += f"<div style='{read_status}margin: 10px 0; padding: 10px; border-left: 4px solid {notification['color']}; background-color: #f9f9f9;'>"
            html += f"{notification['content']}<br>"
            html += f"<small style='color: #777;'>{time_str}</small>"
            html += "</div>"
        
        html += "<button onclick='IPython.notebook.kernel.execute(\"notification_system.mark_all_as_read(); display_notifications()\")'>Mark All as Read</button>"
        html += "</div>"
        return html
    
    def start_simulation(self, interval=5):
        """Start simulating random notifications at intervals"""
        self.running = True
        
        def simulation_worker():
            while self.running:
                # Generate a random event
                event = generate_random_event()
                self.add_notification(event)
                time.sleep(interval)
        
        self.simulation_thread = threading.Thread(target=simulation_worker)
        self.simulation_thread.daemon = True
        self.simulation_thread.start()
    
    def stop_simulation(self):
        """Stop the notification simulation"""
        self.running = False
        if hasattr(self, 'simulation_thread'):
            self.simulation_thread.join(timeout=1)

# Create an instance of the notification system
notification_system = NotificationSystem()

## Display Notifications

Use a loop or callback mechanism to display notifications in real-time or at regular intervals.

In [ ]:
def display_notifications():
    """Display current notifications"""
    notification_system.process_notifications()
    display(HTML(notification_system.generate_html()))

# Function to simulate real-time updates
def start_display_updates(update_interval=3):
    """Start displaying notification updates at intervals"""
    stop_event = threading.Event()
    
    def update_worker():
        while not stop_event.is_set():
            clear_output(wait=True)
            display_notifications()
            time.sleep(update_interval)
    
    update_thread = threading.Thread(target=update_worker)
    update_thread.daemon = True
    update_thread.start()
    
    return stop_event

# Test with some initial notifications
for _ in range(3):
    event = generate_random_event()
    notification_system.add_notification(event)

# Display initial notifications
display_notifications()

In [ ]:
# Start the notification simulation
notification_system.start_simulation(interval=7)  # New notification every ~7 seconds

# Start the display updates
stop_updates = start_display_updates(update_interval=2)  # Update display every 2 seconds

# The simulation will run until you stop it
print("Notification system is now running. New notifications will appear automatically.")
print("Run the next cell when you want to stop the simulation.")

In [ ]:
# Stop the simulation and updates
def stop_everything():
    notification_system.stop_simulation()
    if 'stop_updates' in globals():
        stop_updates.set()
    print("Notification simulation stopped.")

stop_everything()

## Interactive Notification Management

Below are some functions to interact with notifications programmatically.

In [ ]:
# Manually add specific notifications for testing
def add_test_notifications():
    """Add some test notifications of each type"""
    events = [
        FriendRequest("TestUser1", "You"),
        NewMessage("TestUser2", "You", "This is a test message"),
        ForumResponse("TestUser3", "You", "Test Forum")
    ]
    
    for event in events:
        notification_system.add_notification(event)
    
    display_notifications()
    
# Mark specific notification as read
def read_notification(index):
    """Mark a specific notification as read by index (0 is most recent)"""
    notification_system.mark_as_read(index)
    display_notifications()
    
# Functions to create specific notification types
def new_friend_request(sender):
    event = FriendRequest(sender, "You")
    notification_system.add_notification(event)
    display_notifications()

def new_message(sender, message):
    event = NewMessage(sender, "You", message)
    notification_system.add_notification(event)
    display_notifications()
    
def new_forum_response(sender, forum):
    event = ForumResponse(sender, "You", forum)
    notification_system.add_notification(event)
    display_notifications()

# Example usage
# new_friend_request("Manual Sender")
# new_message("Test User", "Hello from a test!")
# new_forum_response("Forum User", "General Discussion")

## Summary

This notification system demonstrates:

1. Different types of notifications (friend requests, messages, forum responses)
2. Real-time notification delivery simulation
3. Notification management (marking as read)
4. Visual display of notifications with appropriate styling

You can extend this system by:
- Adding persistence (saving notifications to a database)
- Implementing notification preferences 
- Adding actions for each notification type (accept/decline friend requests, reply to messages)
- Integrating with actual application events instead of simulations

# Notification System Implementation
This notebook implements a notification system that handles friend requests, new messages, and forum responses.

## Import Required Libraries
Import libraries such as time and random for simulating events, and any other necessary libraries.

In [ ]:
# Import necessary libraries
import time
import random
import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## Define Notification Types
Define the types of notifications (e.g., friend request, new message, forum response) and their structure.

In [ ]:
# Define notification types and their structure

class Notification:
    def __init__(self, user_id, notification_type, content, sender=None, timestamp=None):
        self.id = random.randint(1000, 9999)
        self.user_id = user_id
        self.type = notification_type  # 'friend_request', 'message', 'forum_response'
        self.content = content
        self.sender = sender
        self.timestamp = timestamp if timestamp else datetime.datetime.now()
        self.read = False
        
    def mark_as_read(self):
        self.read = True
        
    def __str__(self):
        return f"[{'✓' if self.read else '✗'}] {self.type.upper()} from {self.sender}: {self.content} ({self.timestamp.strftime('%H:%M:%S')})"
    
    def to_dict(self):
        return {
            'id': self.id,
            'user_id': self.user_id,
            'type': self.type,
            'content': self.content,
            'sender': self.sender,
            'timestamp': self.timestamp,
            'read': self.read
        }

## Simulate User Events
Write code to simulate user events that trigger notifications, such as receiving a friend request or a new message.

In [ ]:
# Simulate user events that trigger notifications

class NotificationSimulator:
    def __init__(self):
        self.users = ["Alice", "Bob", "Charlie", "David", "Emma", "Frank"]
        self.forum_topics = ["Tech Support", "Gaming", "Movies", "Programming", "Travel", "Food"]
        
    def generate_friend_request(self, user_id):
        sender = random.choice([u for u in self.users if u != user_id])
        content = f"{sender} wants to be your friend"
        return Notification(user_id, "friend_request", content, sender)
    
    def generate_message(self, user_id):
        sender = random.choice([u for u in self.users if u != user_id])
        message_templates = [
            "Hey, how are you doing?",
            "Can we meet tomorrow?",
            "Did you see the latest news?",
            "I have a question about our project",
            "Happy birthday!"
        ]
        content = random.choice(message_templates)
        return Notification(user_id, "message", content, sender)
    
    def generate_forum_response(self, user_id):
        sender = random.choice([u for u in self.users if u != user_id])
        topic = random.choice(self.forum_topics)
        content = f"New response in '{topic}' thread"
        return Notification(user_id, "forum_response", content, sender)
    
    def generate_random_notification(self, user_id):
        notification_type = random.choice(["friend_request", "message", "forum_response"])
        if notification_type == "friend_request":
            return self.generate_friend_request(user_id)
        elif notification_type == "message":
            return self.generate_message(user_id)
        else:
            return self.generate_forum_response(user_id)

# Test the simulator
simulator = NotificationSimulator()
current_user = "David"

print("Sample notifications:")
for _ in range(3):
    notification = simulator.generate_random_notification(current_user)
    print(notification)

## Implement Notification System
Create a function or class to handle notifications, including storing and retrieving them.

In [ ]:
# Create a notification system class to handle storing and retrieving notifications

class NotificationSystem:
    def __init__(self, user_id):
        self.user_id = user_id
        self.notifications = []
        self.simulator = NotificationSimulator()
        
    def add_notification(self, notification):
        """Add a notification to the system"""
        if notification.user_id == self.user_id:
            self.notifications.append(notification)
            return True
        return False
    
    def get_all_notifications(self):
        """Get all notifications"""
        return self.notifications
    
    def get_unread_notifications(self):
        """Get only unread notifications"""
        return [n for n in self.notifications if not n.read]
    
    def get_notifications_by_type(self, notification_type):
        """Get notifications filtered by type"""
        return [n for n in self.notifications if n.type == notification_type]
    
    def mark_all_as_read(self):
        """Mark all notifications as read"""
        for notification in self.notifications:
            notification.mark_as_read()
    
    def mark_as_read(self, notification_id):
        """Mark a specific notification as read"""
        for notification in self.notifications:
            if notification.id == notification_id:
                notification.mark_as_read()
                return True
        return False
    
    def clear_notifications(self):
        """Clear all notifications"""
        self.notifications = []
    
    def simulate_new_notifications(self, count=1):
        """Simulate receiving new notifications"""
        for _ in range(count):
            notification = self.simulator.generate_random_notification(self.user_id)
            self.add_notification(notification)
            
    def get_notifications_dataframe(self):
        """Convert notifications to a pandas DataFrame for easier analysis"""
        if not self.notifications:
            return pd.DataFrame()
        return pd.DataFrame([n.to_dict() for n in self.notifications])

# Initialize notification system for current user
notification_system = NotificationSystem(current_user)

# Simulate receiving several notifications
notification_system.simulate_new_notifications(5)

# Display notifications as DataFrame
notification_system.get_notifications_dataframe()

## Display Notifications
Write code to display notifications to the user in a user-friendly format.

In [ ]:
# Create a user interface to display notifications

class NotificationUI:
    def __init__(self, notification_system):
        self.notification_system = notification_system
        
    def display_notification_count(self):
        """Display the count of notifications"""
        total = len(self.notification_system.get_all_notifications())
        unread = len(self.notification_system.get_unread_notifications())
        
        html_content = f"""
        <div style="padding: 10px; background-color: #f0f0f0; border-radius: 5px;">
            <h3>Notifications</h3>
            <p>You have <strong>{total}</strong> total notifications, <strong>{unread}</strong> unread.</p>
        </div>
        """
        display(HTML(html_content))
        
    def display_notifications_list(self):
        """Display a list of all notifications"""
        notifications = self.notification_system.get_all_notifications()
        
        if not notifications:
            display(HTML("<p>No notifications to display.</p>"))
            return
        
        html_content = "<div style='max-height: 300px; overflow-y: auto;'>"
        html_content += "<table style='width: 100%; border-collapse: collapse;'>"
        html_content += "<tr><th>Status</th><th>Type</th><th>Sender</th><th>Content</th><th>Time</th></tr>"
        
        for n in notifications:
            status = "✓" if n.read else "<b>●</b>"
            bgcolor = "#ffffff" if n.read else "#edf7ff"
            html_content += f"""
            <tr style="background-color: {bgcolor}; border-bottom: 1px solid #ddd;">
                <td style="padding: 8px; text-align: center;">{status}</td>
                <td style="padding: 8px;">{n.type.replace('_', ' ').title()}</td>
                <td style="padding: 8px;">{n.sender}</td>
                <td style="padding: 8px;">{n.content}</td>
                <td style="padding: 8px;">{n.timestamp.strftime('%H:%M:%S')}</td>
            </tr>
            """
            
        html_content += "</table></div>"
        display(HTML(html_content))
    
    def create_interactive_ui(self):
        """Create an interactive UI for the notification system"""
        # Create buttons for different actions
        simulate_btn = widgets.Button(description="Generate Notification")
        mark_read_btn = widgets.Button(description="Mark All as Read")
        clear_btn = widgets.Button(description="Clear All")
        refresh_btn = widgets.Button(description="Refresh")
        
        # Filter dropdown
        filter_dropdown = widgets.Dropdown(
            options=['All', 'Unread', 'Friend Requests', 'Messages', 'Forum Responses'],
            value='All',
            description='Filter:',
        )
        
        # Button actions
        def on_simulate_clicked(b):
            self.notification_system.simulate_new_notifications(1)
            self.refresh_display()
            
        def on_mark_read_clicked(b):
            self.notification_system.mark_all_as_read()
            self.refresh_display()
            
        def on_clear_clicked(b):
            self.notification_system.clear_notifications()
            self.refresh_display()
            
        def on_refresh_clicked(b):
            self.refresh_display()
            
        def on_filter_change(change):
            self.refresh_display()
        
        simulate_btn.on_click(on_simulate_clicked)
        mark_read_btn.on_click(on_mark_read_clicked)
        clear_btn.on_click(on_clear_clicked)
        refresh_btn.on_click(on_refresh_clicked)
        filter_dropdown.observe(on_filter_change, names='value')
        
        # Create the UI layout
        display(widgets.HBox([simulate_btn, mark_read_btn, clear_btn, refresh_btn, filter_dropdown]))
        
        # Create an output area for notifications display
        self.output_area = widgets.Output()
        display(self.output_area)
        
        # Initial display
        self.refresh_display()
    
    def refresh_display(self):
        """Refresh the notification display"""
        with self.output_area:
            clear_output(wait=True)
            self.display_notification_count()
            self.display_notifications_list()

# Create and display the notification UI
ui = NotificationUI(notification_system)
ui.create_interactive_ui()

## Testing and Simulation

Let's run a complete simulation of the notification system over time to demonstrate how it might work in a real application.

In [ ]:
# Complete system demonstration with time-based simulation

def run_notification_demo():
    """Run a complete demonstration of the notification system"""
    user = "CurrentUser"
    system = NotificationSystem(user)
    
    print(f"Starting notification demo for user: {user}")
    print("Simulating normal app usage over 1 minute with notifications arriving...")
    
    # Simulate receiving notifications over time
    for i in range(6):
        # Generate between 0-2 notifications each cycle
        num_notifications = random.randint(0, 2)
        if num_notifications > 0:
            system.simulate_new_notifications(num_notifications)
            print(f"[{i*10}s] Received {num_notifications} new notifications!")
        else:
            print(f"[{i*10}s] No new notifications")
            
        # Display current notification status
        unread = len(system.get_unread_notifications())
        if unread > 0:
            print(f"   You have {unread} unread notifications")
            
        # Simulate user sometimes reading notifications
        if random.random() > 0.5 and unread > 0:
            # User checks notifications
            to_read = min(random.randint(1, 3), unread)
            for idx, notification in enumerate(system.get_unread_notifications()):
                if idx < to_read:
                    system.mark_as_read(notification.id)
            print(f"   User checked {to_read} notifications")
            
        # Wait a bit between cycles (reduced for notebook execution)
        time.sleep(1)
    
    # Final status
    print("\nDemo completed! Final notification status:")
    df = system.get_notifications_dataframe()
    
    if len(df) > 0:
        # Summary statistics
        print(f"Total notifications received: {len(df)}")
        print(f"Read notifications: {df['read'].sum()}")
        print(f"Unread notifications: {len(df) - df['read'].sum()}")
        
        # Type breakdown
        type_counts = df['type'].value_counts()
        print("\nNotification types received:")
        for notification_type, count in type_counts.items():
            print(f"  - {notification_type}: {count}")
        
        # Show the dataframe
        return df
    else:
        print("No notifications were generated during the simulation.")
        return pd.DataFrame()

# Run the demonstration
notifications_df = run_notification_demo()
notifications_df

## Conclusion

In this notebook, we've implemented a complete notification system that can:

1. Generate different types of notifications (friend requests, messages, forum responses)
2. Store and manage notifications for a user
3. Mark notifications as read/unread
4. Filter notifications by type
5. Display notifications in a user-friendly format
6. Simulate notification events over time

This system could be integrated into a larger web application by:
- Connecting it to a real database instead of in-memory storage
- Using actual user events from the application instead of simulated ones
- Implementing push notifications or real-time updates using websockets
- Adding user preferences for notification settings

# Notification System Implementation

This notebook implements a notification system for friend requests, new messages, and forum responses.

## Import Required Libraries

In [ ]:
# Import necessary libraries for our notification system
import time
import threading
import queue
import datetime
import json
from IPython.display import display, clear_output
import ipywidgets as widgets

## Define Notification Types

In [ ]:
# Define notification class and types
class Notification:
    def __init__(self, type_name, sender, content, timestamp=None):
        self.type = type_name  # friend_request, message, forum_response
        self.sender = sender
        self.content = content
        self.timestamp = timestamp or datetime.datetime.now()
        self.read = False
        self.id = hash(f"{self.sender}{self.content}{self.timestamp}")
        
    def __str__(self):
        time_str = self.timestamp.strftime("%Y-%m-%d %H:%M:%S")
        status = "✓" if self.read else "●"
        return f"[{status}] {time_str} - {self.type.upper()} from {self.sender}: {self.content}"
    
    def mark_as_read(self):
        self.read = True
        
    def to_dict(self):
        return {
            "id": self.id,
            "type": self.type,
            "sender": self.sender,
            "content": self.content,
            "timestamp": self.timestamp.isoformat(),
            "read": self.read
        }

## Simulate User Events

In [ ]:
# Create a notification manager class
class NotificationManager:
    def __init__(self):
        self.notifications = []
        self.notification_queue = queue.Queue()
        self.subscribers = []
        self._running = False
        self._thread = None
        
    def start(self):
        """Start the notification processor thread"""
        if self._running:
            return
        
        self._running = True
        self._thread = threading.Thread(target=self._process_notifications)
        self._thread.daemon = True
        self._thread.start()
        
    def stop(self):
        """Stop the notification processor thread"""
        self._running = False
        if self._thread:
            self._thread.join(timeout=1)
            
    def _process_notifications(self):
        """Process notifications from the queue"""
        while self._running:
            try:
                notification = self.notification_queue.get(timeout=1)
                self.notifications.append(notification)
                
                # Notify subscribers
                for subscriber in self.subscribers:
                    subscriber(notification)
                    
                self.notification_queue.task_done()
            except queue.Empty:
                pass
    
    def add_subscriber(self, callback):
        """Add a subscriber to be notified when new notifications arrive"""
        self.subscribers.append(callback)
        
    def remove_subscriber(self, callback):
        """Remove a subscriber"""
        if callback in self.subscribers:
            self.subscribers.remove(callback)
    
    def send_friend_request(self, sender, recipient):
        """Simulate sending a friend request"""
        notification = Notification(
            "friend_request", 
            sender, 
            f"sent you a friend request"
        )
        self.notification_queue.put(notification)
        return notification
        
    def send_message(self, sender, recipient, message):
        """Simulate sending a message"""
        notification = Notification(
            "message",
            sender,
            f"sent you a message: '{message}'"
        )
        self.notification_queue.put(notification)
        return notification
        
    def send_forum_response(self, sender, post_id, response):
        """Simulate a forum response"""
        notification = Notification(
            "forum_response",
            sender,
            f"responded to your post #{post_id}: '{response}'"
        )
        self.notification_queue.put(notification)
        return notification
    
    def get_unread_count(self):
        """Get the count of unread notifications"""
        return sum(1 for n in self.notifications if not n.read)
    
    def mark_as_read(self, notification_id):
        """Mark a notification as read"""
        for notification in self.notifications:
            if notification.id == notification_id:
                notification.mark_as_read()
                return True
        return False
    
    def mark_all_as_read(self):
        """Mark all notifications as read"""
        for notification in self.notifications:
            notification.mark_as_read()
            
    def get_notifications(self, unread_only=False):
        """Get all notifications or only unread ones"""
        if unread_only:
            return [n for n in self.notifications if not n.read]
        return self.notifications

## Implement Notification System

In [ ]:
# Initialize the notification manager
notification_manager = NotificationManager()
notification_manager.start()

# Function to simulate user events
def simulate_user_events():
    """Simulate user events to generate notifications"""
    notification_manager.send_friend_request("Alice", "User")
    time.sleep(2)
    
    notification_manager.send_message("Bob", "User", "Hey! How are you doing?")
    time.sleep(3)
    
    notification_manager.send_forum_response("Charlie", "12345", "Great point you made there!")
    time.sleep(1)
    
    notification_manager.send_friend_request("David", "User")
    time.sleep(2)
    
    notification_manager.send_message("Alice", "User", "Did you get my friend request?")

# Callback function for new notifications
def on_new_notification(notification):
    """Callback function for when new notifications arrive"""
    print(f"New notification received: {notification}")

# Add the subscriber
notification_manager.add_subscriber(on_new_notification)

## Display Notifications

In [ ]:
# Function to display notifications in a user-friendly format
def display_notifications(unread_only=False):
    """Display notifications in a formatted way"""
    clear_output(wait=True)
    
    notifications = notification_manager.get_notifications(unread_only)
    
    if not notifications:
        print("No notifications" + (" (unread)" if unread_only else ""))
        return
    
    unread_count = notification_manager.get_unread_count()
    print(f"Notifications ({unread_count} unread):")
    print("-" * 70)
    
    for i, notification in enumerate(notifications, 1):
        print(f"{i}. {notification}")
    
    print("-" * 70)

# Create widgets for interactive display
def create_notification_ui():
    """Create interactive UI for notifications"""
    display_btn = widgets.Button(description="Display All")
    unread_btn = widgets.Button(description="Show Unread")
    mark_read_btn = widgets.Button(description="Mark All Read")
    simulate_btn = widgets.Button(description="Simulate Events")
    
    def on_display_btn_click(b):
        display_notifications(unread_only=False)
    
    def on_unread_btn_click(b):
        display_notifications(unread_only=True)
        
    def on_mark_read_btn_click(b):
        notification_manager.mark_all_as_read()
        display_notifications()
        
    def on_simulate_btn_click(b):
        simulate_user_events()
        display_notifications()
    
    display_btn.on_click(on_display_btn_click)
    unread_btn.on_click(on_unread_btn_click)
    mark_read_btn.on_click(on_mark_read_btn_click)
    simulate_btn.on_click(on_simulate_btn_click)
    
    return widgets.HBox([display_btn, unread_btn, mark_read_btn, simulate_btn])

In [ ]:
# Display the notification UI
ui = create_notification_ui()
display(ui)

# Initialize with some notifications
print("Starting notification system. Click 'Simulate Events' to generate notifications.")

## Example: Testing the Notification System

In [ ]:
# Run this cell to test the notification system with a series of events
def test_notification_system():
    # Generate some test notifications
    print("Generating test notifications...")
    
    # Friend requests
    notification_manager.send_friend_request("Jane", "User")
    notification_manager.send_friend_request("Mike", "User")
    
    # Messages
    notification_manager.send_message("Sarah", "User", "Hello! Are we meeting today?")
    notification_manager.send_message("John", "User", "Check out this new feature I built!")
    
    # Forum responses
    notification_manager.send_forum_response("Alex", "7890", "I have the same issue!")
    notification_manager.send_forum_response("Kelly", "5432", "Thanks for sharing your solution")
    
    # Display the notifications
    display_notifications()
    
    print("\nTest complete. You can now interact with the notification system using the buttons above.")

# Uncomment the line below to run the test
# test_notification_system()

## Conclusion

This notebook demonstrates a simple notification system that can handle different types of notifications including friend requests, messages, and forum responses. The system uses threading to process notifications in the background and provides a simple user interface to interact with notifications.

Key features implemented:
- Different notification types with appropriate structure
- Real-time notification processing using threading
- Notification queue to handle asynchronous events
- Marking notifications as read/unread
- Display functions for easy visualization
- Interactive UI using ipywidgets

This system can be expanded to include features like notification preferences, filters, and integration with a real backend system.